In [1]:
import pandas as pd
import numpy as np

from statsmodels.tsa.vector_ar.var_model import VAR
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.deterministic import DeterministicProcess

In [4]:
ts_df = pd.read_csv('C:\\Users\\YUE\\Desktop\\python_work\\heavy\\sale_bypart.csv',index_col=0)
ts_df.index = pd.to_datetime(ts_df.index, format="%Y%m")
print('数据形状：',ts_df.shape)
ts_df.head()

数据形状： (30, 1200)


,id_1,id_2,id_3,id_4,id_5,id_6,id_7,id_8,id_9,id_10,...,id_1191,id_1192,id_1193,id_1194,id_1195,id_1196,id_1197,id_1198,id_1199,id_1200
month,,,,,,,,,,,,,,,,,,,,,
2018-01-01,1.0,16.0,6.0,0.0,22.0,64.0,0.0,0.0,10.0,10.0,...,9.0,0.0,4.0,95.0,0.0,1.0,2.0,0.0,26.0,0.0
2018-02-01,1.0,35.0,2.0,0.0,18.0,8.0,0.0,0.0,0.0,14.0,...,16.0,0.0,5.0,16.0,0.0,1.0,0.0,0.0,24.0,0.0
2018-03-01,0.0,13.0,10.0,0.0,24.0,111.0,0.0,4.0,0.0,10.0,...,28.0,0.0,1.0,75.0,1.0,3.0,1.0,0.0,46.0,0.0
2018-04-01,5.0,27.0,17.0,0.0,17.0,122.0,0.0,0.0,10.0,9.0,...,29.0,0.0,5.0,107.0,2.0,2.0,3.0,0.0,64.0,0.0
2018-05-01,2.0,45.0,4.0,4.0,36.0,100.0,0.0,8.0,0.0,19.0,...,47.0,0.0,6.0,150.0,2.0,0.0,1.0,0.0,22.0,1.0


In [33]:
# 评价函数: 根据任务要求可以定义评价函数用于本地验证
def get_score(pdf,rdf):
    score_df = (pdf-rdf).abs()/rdf <= 0.3
    score = score_df.mean(axis=1).round(4)
    print(score*100)
    print('accurancy:',round(score.mean()*100,2))

### 本地加 验证后发现VAR与LR模型以
- 第1个月使用VAR模型，第2-3月使用LR模型
- 由于零件中有很多的销量非常稀疏，因此做一步处理：将48个月均销量小于1的零件销量直接调整为0

In [6]:
# VAR
v_model = VAR(ts_df)
v_model_fit = v_model.fit(3)
lag_order = v_model_fit.k_ar

# forecast for the 1st month
y_fore1 = v_model_fit.forecast(ts_df.values[-lag_order:], 1)
y_fore1[y_fore1 < 0] = 0

C:\Users\YUE\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'


In [8]:
# LR model
y = ts_df.copy()

# Create trend features
dp = DeterministicProcess(
    index=y.index,  # dates from the training data
    constant=True,  # the intercept
    order=2,        # quadratic trend
    drop=True,      # drop terms to avoid collinearity
)
X = dp.in_sample()  # features for the training data
X_fore = dp.out_of_sample(steps=3)

X['mon']=X.index.month
X_fore.set_index([pd.to_datetime(['2020-7-1','2020-8-1','2020-9-1'])],inplace=True)
X_fore['mon']=X_fore.index.month

# Fit trend model
model = LinearRegression(fit_intercept=False)
model.fit(X, y)

y_fore_l = pd.DataFrame(
    model.predict(X_fore),
    index=X_fore.index,
    columns=y.columns,
)
y_fore_l[y_fore_l<0] = 0

C:\Users\YUE\anaconda3\lib\site-packages\statsmodels\tsa\deterministic.py:152: UserWarning: Only PeriodIndexes, DatetimeIndexes with a frequency set, RangesIndexes, and NumericIndex with a unit increment support extending. The index is set will contain the position relative to the data length.
  warnings.warn(


In [24]:
avg_num = (ts_df.sum()/30)
sparse_id = ts_df.columns[avg_num<1]

In [20]:
sub = pd.read_csv('C:\\Users\\YUE\\Desktop\\python_work\\heavy\\sub.csv')
sub_wide = sub.pivot_table(index='month',columns=['matrl_id'])
sub_wide.index = pd.to_datetime(sub_wide.index, format="%Y%m")
sub_wide.columns = ts_df.columns

sub_wide.iloc[0] = y_fore1[0]
sub_wide.iloc[1:] = y_fore_l.iloc[1:].values

sub['nums'] = sub_wide.stack().squeeze().reset_index(drop=True).round(0)
# sub.loc[sub.matrl_id.isin(sparse_id),'nums']= 0
sub.to_csv('sub_best.csv',index=False)

In [21]:
tr = pd.read_csv('C:\\Users\\YUE\\Desktop\\python_work\\heavy\\sub_v13.csv')

In [25]:
(sub==tr).sum()

matrl_id    3600
month       3600
nums        1855
dtype: int64